<a href="https://colab.research.google.com/github/NanduRR/data/blob/main/Copy_of_Nandu_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=42c34c44cb05d1191e0b76342320b0e7b0cc4f4579a8cf82ea607234305a0e90
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.4/640.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 94.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable  HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [ ]:
from haystack.utils import launch_es
launch_es()

In [ ]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/build_a_scalable_question_answering_system"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip", 
    output_dir=doc_dir
)


INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip to 'data/build_a_scalable_question_answering_system'


True

In [ ]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

In [ ]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/183 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/183 [00:00<?, ?docs/s]

{'documents': [<Document: {'content': "\n\n'''''A Game of Thrones''''' is a role-playing game produced by Guardians of Order based on the ''A Song of Ice and Fire'' fantasy series by George R. R. Martin.\n\n==Description==\nThe game is designed to be usable with two RPG systems: the d20 System and the Tri-Stat dX system. Two editions were made: a serial-numbered edition limited to 2500 copies, ; and a standard edition, . The limited edition is faux-leather bound with silver gilt pages and includes rules for both systems, and includes an interview with Martin. The standard edition contains only the d20 system rules. The book was created by Guardians of Order and released by Sword & Sorcery, a subsidiary of White Wolf Games.\n\n==Reception==\nThe ''A Game of Thrones'' RPG (AGOT RPG) was nominated for several ENnie Awards and won 2006 awards for: Best Production (Silver), Best Game (Silver), and Best d20/OGL Product (Silver).\n\n==History==\nOn 28 July 2006, Martin announced that he had r

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

In [ ]:
prediction = querying_pipeline.run(
    query="what received widespread critical acclaim?",
    params={
        "Retriever": {"top_k": 1},
        "Reader": {"top_k": 1}
    }
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [ ]:
from pprint import pprint

#pprint(prediction)
#a=pprint(prediction['answers'][0])
b=prediction['answers'][0]
pprint(b.answer)



'The Spoils of War'


In [ ]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium` and `all`
)



Query: what received widespread critical acclaim?
Answers:
[   {   'answer': 'The Spoils of War',
        'context': 'ceived 1.022 million timeshift viewers.\n'
                   '\n'
                   '===Critical reception===\n'
                   '"The Spoils of War" received widespread acclaim from '
                   'critics, who listed the battle '}]


In [ ]:
port_no = 5000
from flask import Flask, request, jsonify


In [ ]:
app = Flask(__name__)
ngrok.set_auth_token("2M4WFe5SqZudJNv7V7gGYUYL5om_o62JGLSsYaLnLqF93oia")
public_url =  ngrok.connect(port_no).public_url


@app.route("/")
def home():
    return f"Running Flask on Google Colab!"

@app.route('/webhook',methods=['POST'])
def dialogflow_webhook():
    req = request.get_json()
    user_input_text = req['queryResult']['queryText']
    if (user_input_text.split(':')[0]="Data" or user_input_text.split(':')[0]="data"):
      f = open("data.txt", "w")
      f.write(user_input_text.split(':')[1])
    else:
      prediction = querying_pipeline.run(
      query=user_input_text.split(':')[1],
      params={
          "Retriever": {"top_k": 1},
          "Reader": {"top_k": 1}
      }
      )
      from pprint import pprint

      pprint(prediction)
      a=pprint(prediction['answers'][0])
      b=prediction['answers'][0]
    
      response_text = str(b.answer)
      response = {
          "fulfillmentText": response_text
      }
      return jsonify(response)


print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)

To acces the Gloable link please click http://2a68-34-125-203-250.ngrok.io
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

INFO:werkzeug:127.0.0.1 - - [08/Mar/2023 21:57:27] "POST /webhook HTTP/1.1" 200 -


{'answers': [<Answer {'answer': 'Lysa Arryn', 'type': 'extractive', 'score': 0.08046305179595947, 'context': "d Sansa depart King's Landing for the Eyrie, home of Catelyn's sister Lysa Arryn.\n\nAfter Balon, Robb and Joffrey die, as Melisandre predicted, Davos h", 'offsets_in_document': [{'start': 680, 'end': 690}], 'offsets_in_context': [{'start': 70, 'end': 80}], 'document_ids': ['5d21bd823c0e04d50fbdc6643c6cbf3c'], 'meta': {'_split_id': 7}}>],
 'documents': [<Document: {'content': 'Joffrey\'s grandfather Tywin Lannister, the Hand of the King, compels Sansa to marry his dwarf son Tyrion, to enable Lannister control of the North; but Tyrion refuses to consummate the marriage against her will.\n\nMargaery and Joffrey\'s wedding is held as planned, but during the wedding feast, Joffrey is poisoned and dies. Tyrion is accused of the murder by his sister Cersei, Joffrey\'s mother, and arrested.  Sansa escapes the castle with the help of Lord Petyr "Littlefinger" Baelish, who admits to her

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

INFO:werkzeug:127.0.0.1 - - [08/Mar/2023 22:16:52] "POST /webhook HTTP/1.1" 200 -


{'answers': [<Answer {'answer': 'Thor: The Dark World', 'type': 'extractive', 'score': 0.0039580948650836945, 'context': "s a director for several big budget Hollywood films, including ''Thor: The Dark World'' and ''Terminator Genisys''. In an interview with ''Entertainme", 'offsets_in_document': [{'start': 438, 'end': 458}], 'offsets_in_context': [{'start': 65, 'end': 85}], 'document_ids': ['50763b0a82fcd6767092bbb110128ce2'], 'meta': {'_split_id': 8}}>],
 'documents': [<Document: {'content': 'This was Taylor\'s seventh episode as a director for the series, but it was his first episode since the second season, where he directed that season\'s finale episode, "Valar Morghulis". He was also a director for two episodes in the first season, "Baelor" and "Fire and Blood", as well as four other episodes in the second season. Since his hiatus from the series, Taylor was a director for several big budget Hollywood films, including \'\'Thor: The Dark World\'\' and \'\'Terminator Genisys\'\'. I

In [ ]:
print(str(b))

<Answer: answer='The Spoils of War', score=0.5198479890823364, context='ceived 1.022 million timeshift viewers.

===Critic...'>
